# 🏆 [Step-by-Step] 모델 성능 평가 파이프라인

이 노트북은 학습 중에 생성된 **Denoising 모델**과 **Deconvolution 모델** 체크포인트(`.ckpt`) 파일 쌍의 최종 성능을 평가하기 위한 전용 파이프라인입니다.

**실행 방법:**
1. **"1. 환경 설정"** 셀을 실행하여 Google Drive를 마운트하고 프로젝트 경로를 설정합니다. (최초 1회)
2. **"2. 평가 실행"** 셀에서 평가하고 싶은 **`DENOISING_CKPT_PATH`**와 **`DECONV_CKPT_PATH`** 변수 값을 모두 수정합니다.
3. **"2. 평가 실행"** 셀을 실행하면, 아래 과정이 자동으로 진행됩니다.
    - `create_evaluation_results_sbs.py`가 실행되어 두 모델을 순차적으로 통과시킨 최종 복원 이미지를 `result` 폴더에 `.npy` 파일로 저장합니다.
    - `evaluate.ipynb`의 핵심 로직이 실행되어 `result` 폴더와 `label` 폴더를 비교하고 최종 PSNR/SSIM 점수를 출력합니다.


### 1. 환경 설정


In [ ]:
# @title Google Drive 마운트 및 Git 동기화
import os
import sys
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive', force_remount=True)

# 💥 사용자 설정: 자신의 Google Drive 프로젝트 경로에 맞게 수정해주세요.
PROJECT_ROOT = "/content/drive/MyDrive/Data Scientist/Project/Week5/week5"

# 작업 디렉토리 변경
os.chdir(PROJECT_ROOT)
print(f"Current working directory: {os.getcwd()}")

# --- Git 동기화 ---
print("\nSyncing with GitHub Repository...")
# 노트북 파일의 로컬 변경사항을 폐기하여 충돌 방지
!git checkout -- run_evaluation.ipynb
# 원격 저장소의 최신 코드를 받아옴
!git pull origin main
print("--- Git sync complete ---\n")


# 경로 추가 및 라이브러리 설치
sys.path.append(PROJECT_ROOT)
print("Installing required libraries...")
%pip install loguru tqdm --quiet
print("Libraries installed.")

print("Environment setup complete.")


### 2. 평가 실행


In [ ]:
# @title 체크포인트 경로 지정 및 최종 평가 실행
import shutil
from pathlib import Path
import sys
import os

# --- ⚙️ 1. 사용자 설정: 평가할 체크포인트 경로를 모두 입력하세요. ---
# 예시: "logs_sbs_denoising_dncnn/00000_train/checkpoints/checkpoint_best.ckpt"
DENOISING_CKPT_PATH = "" #@param {type:"string"}
# 예시: "logs_sbs_deconv_unet/00001_train/checkpoints/checkpoint_best.ckpt"
DECONV_CKPT_PATH = "" #@param {type:"string"}
# -------------------------------------------------------------------

# --- 자동으로 경로 및 폴더 설정 ---
RESULT_FOLDER = "result"

# 이전 결과가 있다면 삭제
if Path(RESULT_FOLDER).exists():
    print(f"Removing old '{RESULT_FOLDER}' directory...")
    shutil.rmtree(RESULT_FOLDER)

# --- 1단계: 결과물(.npy 파일) 생성 ---
print("\n[Phase 1/2] Creating result files from checkpoint...")

if DENOISING_CKPT_PATH and DECONV_CKPT_PATH:
    print("Running in Step-by-Step mode...")
    !python "create_evaluation_results_sbs.py" \
        --denoising_ckpt_path "{DENOISING_CKPT_PATH}" \
        --deconv_ckpt_path "{DECONV_CKPT_PATH}" \
        --result_dir "{RESULT_FOLDER}"
else:
    print("🚨 Error: Please provide paths for BOTH denoising and deconvolution checkpoints.")

# --- 2단계: 최종 점수 계산 (결과물이 생성된 경우에만 실행) ---
if Path(RESULT_FOLDER).exists() and any(Path(RESULT_FOLDER).iterdir()):
    print("\n[Phase 2/2] Calculating final score with evaluate module...")
    
    original_dir = os.getcwd()
    dataset_dir = os.path.join(original_dir, "dataset")

    try:
        os.chdir(dataset_dir)
        !python setup.py build_ext --inplace --quiet

        if '_evaluate' in sys.modules:
            del sys.modules['_evaluate']
        if 'dataset._evaluate' in sys.modules:
            del sys.modules['dataset._evaluate']
        
        import _evaluate
        
        evaluator = _evaluate.Evaluator(data_root=f"../{RESULT_FOLDER}", label_root="label")
        evaluator()
    except Exception as e:
        print(f"An error occurred during evaluation: {e}")
    finally:
        os.chdir(original_dir)
        print(f"\nReturned to original directory: {os.getcwd()}")
else:
    print("\n[Phase 2/2] Skipped. No result files were generated in Phase 1.")

